# Optical Flow

----
#### NOTE: It is probably a good idea to restart the kernel if you ever run these cells, as the tracking algo can sometimes get caught in a loop with your camera.

## Lucas-Kanade Optical Flow

In [1]:
import numpy as np
import cv2 

In [2]:
# Parameters for ShiTomasi corner detection (good features to track paper)
corner_track_params = dict(maxCorners = 10,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

### Parameters for Lucas Kanade Optical Flow

Detect the motion of specific points or the aggregated motion of regions by modifying the winSize argument. This determines the integration window size. Small windows are more sensitive to noise and may miss larger motions. Large windows will “survive” an occlusion.

The integration appears smoother with the larger window size.

criteria has two here - the max number (10 above) of iterations and epsilon (0.03 above). More iterations means a more exhaustive search, and a smaller epsilon finishes earlier. These are primarily useful in exchanging speed vs accuracy, but mainly stay the same.

When maxLevel is 0, it is the same algorithm without using pyramids (ie, calcOpticalFlowLK). Pyramids allow finding optical flow at various resolutions of the image. 

In [3]:
# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (200,200),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10,0.03))

In [4]:
# Capture the video
cap = cv2.VideoCapture(0)

# Grab the very first frame of the stream
ret, prev_frame = cap.read()

# Grab a grayscale image (We will refer to this as the previous frame)
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

# Grabbing the corners
prevPts = cv2.goodFeaturesToTrack(prev_gray, mask = None, **corner_track_params)

# Create a matching mask of the previous frame for drawing on later
mask = np.zeros_like(prev_frame)


while True:
    
    # Grab current frame
    ret,frame = cap.read()
    
    # Grab gray scale
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Calculate the Optical Flow on the Gray Scale Frame
    nextPts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, frame_gray, prevPts, None, **lk_params)
    
    # Using the returned status array (the status output)
    # status output status vector (of unsigned chars); each element of the vector is set to 1 if
    # the flow for the corresponding features has been found, otherwise, it is set to 0.
    good_new = nextPts[status==1]
    good_prev = prevPts[status==1]
    
    # Use ravel to get points to draw lines and circles
    for i,(new,prev) in enumerate(zip(good_new,good_prev)):
        
        x_new,y_new = new.ravel()
        x_prev,y_prev = prev.ravel()
        
        # Lines will be drawn using the mask created from the first frame
        mask = cv2.line(mask, (x_new,y_new),(x_prev,y_prev), (0,255,0), 3)
        
        # Draw red circles at corner points
        frame = cv2.circle(frame,(x_new,y_new),8,(0,0,255),-1)
    
    # Display the image along with the mask we drew the line on.
    img = cv2.add(frame,mask)
    cv2.imshow('frame',img)
    
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
   
    # Now update the previous frame and previous points
    prev_gray = frame_gray.copy()
    prevPts = good_new.reshape(-1,1,2)
    
    
cv2.destroyAllWindows()
cap.release()

# Detailed Explanation of Optical Flow Code

---

## **1. Shi-Tomasi Corner Detection**

### Code:
```python
prevPts = cv2.goodFeaturesToTrack(prev_gray, mask=None, **corner_track_params)
```

- **Concept**:  
  Shi-Tomasi is a corner detection algorithm that finds "good features to track." Corners are points in the image where the intensity changes significantly in both directions (horizontal and vertical), making them easy to identify across frames.

- **Why Corners?**  
  - Corners are more stable and easier to track compared to flat regions or edges:
    - A flat region (e.g., plain wall) looks similar in all directions, making it hard to track.
    - An edge only has a change in one direction, so it’s less distinctive.
    - A corner has a unique intensity change in all directions.

- **Parameters Explained**:
  - `maxCorners`: Limits the number of corners detected (e.g., 10 corners).
  - `qualityLevel`: A threshold to discard weak corners. It’s a fraction of the strongest corner's quality. Higher values result in fewer, but stronger corners.
  - `minDistance`: Ensures corners are not too close together.
  - `blockSize`: Size of the window used to calculate the intensity gradient.

- **How it Works**:
  1. Computes the gradient (intensity change) in the image.
  2. Looks for pixels where the gradient is strong in both directions.
  3. Applies the `qualityLevel` and `minDistance` filters.

- **Example Output**:  
  An array of corner points, each with \((x, y)\) coordinates. These points will be used for tracking in the next frame.

---

## **2. Lucas-Kanade Optical Flow**

### Code:
```python
nextPts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, frame_gray, prevPts, None, **lk_params)
```

- **Concept**:  
  The Lucas-Kanade (LK) method tracks points across frames. It assumes that:
  1. The motion between two frames is small.
  2. Nearby points move in a similar way.

  The algorithm finds the new position of each corner point in the next frame.

- **How It Works**:
  1. A small window (defined by `winSize`) is placed around each point to track.
  2. The algorithm looks for the most similar region in the next frame by comparing brightness patterns inside the window.
  3. It iteratively adjusts the position until the difference is minimized (controlled by `criteria`).

- **Parameters Explained**:
  - `prev_gray`: Previous frame (grayscale).
  - `frame_gray`: Current frame (grayscale).
  - `prevPts`: Points to track in the previous frame.
  - `winSize`: Size of the search window for matching points.
  - `maxLevel`: Number of image pyramid levels for multi-scale tracking. Higher levels allow tracking across varying sizes (e.g., zoomed-in/out objects).
  - `criteria`: Stopping conditions for iterations:
    - `TERM_CRITERIA_EPS`: Stops when the positional change is small (e.g., < 0.03).
    - `TERM_CRITERIA_COUNT`: Stops after a fixed number of iterations (e.g., 10).

- **Outputs**:
  - `nextPts`: Estimated new positions of the points in the current frame.
  - `status`: A binary array where `1` means the point was successfully tracked, and `0` means it wasn’t.
  - `err`: Error values indicating how well the point was tracked.

---

## **3. Tracking Points Across Frames**

### Code:
```python
good_new = nextPts[status == 1]
good_prev = prevPts[status == 1]
```

- **Concept**:  
  Not all points are successfully tracked between frames.  
  - `status == 1` filters only the points that were successfully tracked.  
    - `good_prev`: The positions of the tracked points in the previous frame.  
    - `good_new`: The positions of the same points in the current frame.

---

## **4. Drawing the Motion**

### Code:
```python
for i, (new, prev) in enumerate(zip(good_new, good_prev)):
    x_new, y_new = new.ravel()
    x_prev, y_prev = prev.ravel()
    mask = cv2.line(mask, (x_new, y_new), (x_prev, y_prev), (0, 255, 0), 3)
    frame = cv2.circle(frame, (x_new, y_new), 8, (0, 0, 255), -1)
```

- **What It Does**:
  - Loops through each pair of points (previous and current positions).
  - Draws a **line** on the `mask` from the previous position to the current position to represent motion.
  - Draws a **red circle** on the `frame` at the current position.

- **Key Details**:
  - `zip(good_new, good_prev)`: Combines the current and previous points so they can be processed together.
  - `ravel()`: Flattens the \((x, y)\) coordinates into simple numbers for easier drawing.
  - `cv2.line`: Draws the green motion line on the mask.
    - `(0, 255, 0)`: Green color.
    - `3`: Thickness of the line.
  - `cv2.circle`: Draws a red dot at the new position of the point.
    - `(0, 0, 255)`: Red color.
    - `-1`: Fills the circle.

---

## **5. Combining Mask and Frame**

### Code:
```python
img = cv2.add(frame, mask)
cv2.imshow('frame', img)
```

- **Concept**:  
  Combines the motion lines (on `mask`) with the current frame (`frame`) and displays the result:
  - `cv2.add`: Adds the two images together.
  - `cv2.imshow`: Displays the combined image in a window called `'frame'`.

---

## **6. Loop Control and Update**

### Key Updates:
```python
k = cv2.waitKey(30) & 0xff
if k == 27:
    break
```

- **`cv2.waitKey`**: Waits for 30 ms to check if a key is pressed.  
  - `27`: ASCII code for the `Esc` key. If pressed, the loop exits.

### Updating Variables:
```python
prev_gray = frame_gray.copy()
prevPts = good_new.reshape(-1, 1, 2)
```

- **Why Update?**  
  To track motion, the current frame becomes the new "previous frame" for the next iteration.
  - `prev_gray`: The current grayscale frame is stored as the previous frame.
  - `prevPts`: The new points (`good_new`) are reshaped into the format required for tracking in the next loop.

---

## **7. Cleanup**

### Code:
```python
cv2.destroyAllWindows()
cap.release()
```

- **Purpose**:
  - `cv2.destroyAllWindows()`: Closes all OpenCV windows.
  - `cap.release()`: Releases the video capture resource, freeing up the camera for other applications.

---

## **Summary of Workflow**
1. Detect key points (corners) in the first frame.
2. Continuously read new frames and convert them to grayscale.
3. Use the Lucas-Kanade method to track the motion of these key points.
4. Draw lines and circles to visualize the motion.
5. Update frames and points to repeat the process.
6. Exit and clean up resources when the user presses the `Esc` key.

# Dense Optical Flow in OpenCV

calcOpticalFlowFarneback(prev, next, flow, pyr_scale, levels, winsize, iterations, poly_n, poly_sigma, flags) -> flow

This function computes a dense optical flow using the Gunnar Farneback's algorithm.

Here are the parameters for the function and what they represent:
   

* prev first 8-bit single-channel input image.
* next second input image of the same size and the same type as prev.
* flow computed flow image that has the same size as prev and type CV_32FC2.
* pyr_scale parameter, specifying the image scale (\<1) to build pyramids for each image
    * pyr_scale=0.5 means a classical pyramid, where each next layer is twice smaller than the previous one.
    
* levels number of pyramid layers including the initial image; levels=1 means that no extra layers are created and only the original images are used.
* winsize averaging window size
    * larger values increase the algorithm robustness to image
* noise and give more chances for fast motion detection, but yield more blurred motion field.
* iterations number of iterations the algorithm does at each pyramid level.
* poly_n size of the pixel neighborhood used to find polynomial expansion in each pixel
    * larger values mean that the image will be approximated with smoother surfaces, yielding more robust algorithm and more blurred motion field, typically poly_n =5 or 7.
* poly_sigma standard deviation of the Gaussian that is used to smooth derivatives used as a basis for the polynomial expansion; for poly_n=5, you can set poly_sigma=1.1, for poly_n=7, a good value would be poly_sigma=1.5.

In [ ]:
import cv2 
import numpy as np

# Capture the frame
cap = cv2.VideoCapture(0)
ret, frame1 = cap.read()

# Get gray scale image of first frame and make a mask in HSV color
prvsImg = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)

hsv_mask = np.zeros_like(frame1)
hsv_mask[:,:,1] = 255

while True:
    ret, frame2 = cap.read()
    nextImg = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
    
    # Check out the markdown text above for a break down of these paramters, most of these are just suggested defaults
    flow = cv2.calcOpticalFlowFarneback(prvsImg,nextImg, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    
    
    # Color the channels based on the angle of travel
    # Pay close attention to your video, the path of the direction of flow will determine color!
    mag, ang = cv2.cartToPolar(flow[:,:,0], flow[:,:,1],angleInDegrees=True)
    hsv_mask[:,:,0] = ang/2
    hsv_mask[:,:,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    
    # Convert back to BGR to show with imshow from cv
    bgr = cv2.cvtColor(hsv_mask,cv2.COLOR_HSV2BGR)
    cv2.imshow('frame2',bgr)
    
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    
    # Set the Previous image as the next iamge for the loop
    prvsImg = nextImg

    
cap.release()
cv2.destroyAllWindows()

# Explanation of Optical Flow Using Farneback Method

---

## **1. Importing Required Libraries**

### Code:
```python
import cv2 
import numpy as np
```

- **`cv2`**: OpenCV library for image and video processing.
- **`numpy`**: For numerical operations and creating masks.

---

## **2. Setting Up Video Capture**

### Code:
```python
cap = cv2.VideoCapture(0)
ret, frame1 = cap.read()
```

- **`cv2.VideoCapture(0)`**: Captures video from the default camera (camera index `0`).
- **`cap.read()`**:
  - `ret`: Boolean indicating if the frame was successfully captured.
  - `frame1`: First frame of the video stream.

---

## **3. Preparing the First Frame**

### Code:
```python
prvsImg = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
hsv_mask = np.zeros_like(frame1)
hsv_mask[:,:,1] = 255
```

- **Convert to Grayscale**:
  - `cv2.cvtColor`: Converts the first frame (`frame1`) from BGR to grayscale.
  - **Why?** Grayscale simplifies optical flow calculations by reducing color complexity.

- **Create HSV Mask**:
  - `np.zeros_like(frame1)`: Creates a blank array (same shape as `frame1`) initialized to zero.
  - `hsv_mask[:,:,1] = 255`: Sets the saturation channel (`S`) to maximum for vivid coloring when converted to BGR.

---

## **4. Loop Through Frames**

### Code:
```python
while True:
    ret, frame2 = cap.read()
    nextImg = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
```

- Captures the next frame (`frame2`) and converts it to grayscale (`nextImg`).
- **Purpose**: To compute the optical flow between the current frame (`prvsImg`) and the next frame (`nextImg`).

---

## **5. Computing Optical Flow**

### Code:
```python
flow = cv2.calcOpticalFlowFarneback(prvsImg, nextImg, None, 0.5, 3, 15, 3, 5, 1.2, 0)
```

- **Method**: `cv2.calcOpticalFlowFarneback` calculates dense optical flow using the Farneback algorithm.
- **Parameters**:
  - `prvsImg`: Previous grayscale image.
  - `nextImg`: Current grayscale image.
  - `None`: Placeholder for optional flow initialization.
  - `0.5`: Pyramid scale factor (reduces image size to capture large motions).
  - `3`: Number of pyramid levels (controls motion detail).
  - `15`: Window size for motion averaging.
  - `3`: Number of iterations at each pyramid level.
  - `5`: Pixel neighborhood size for polynomial expansion.
  - `1.2`: Standard deviation for Gaussian filtering.
  - `0`: Flags (default).

- **Output**: A 2D vector field (`flow`) representing motion between the two frames.
  - `flow[:,:,0]`: Horizontal motion.
  - `flow[:,:,1]`: Vertical motion.

---

## **6. Extracting Magnitude and Angle**

### Code:
```python
mag, ang = cv2.cartToPolar(flow[:,:,0], flow[:,:,1], angleInDegrees=True)
```

- **`cv2.cartToPolar`**: Converts Cartesian motion vectors (\(dx, dy\)) to polar coordinates:
  - `mag`: Magnitude of motion (speed).
  - `ang`: Angle of motion (direction) in degrees.

---

## **7. Coloring the Optical Flow**

### Code:
```python
hsv_mask[:,:,0] = ang / 2
hsv_mask[:,:,2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
```

- **Hue (Angle)**:
  - `hsv_mask[:,:,0]`: Represents the angle of motion (divided by 2 to fit HSV range of 0-180 degrees).
- **Value (Magnitude)**:
  - `cv2.normalize`: Scales the magnitude values (`mag`) to fit in the range [0, 255] for proper visualization.

---

## **8. Converting Back to BGR**

### Code:
```python
bgr = cv2.cvtColor(hsv_mask, cv2.COLOR_HSV2BGR)
cv2.imshow('frame2', bgr)
```

- **`cv2.cvtColor`**: Converts the HSV mask to BGR for display.
- **`cv2.imshow`**: Displays the colored optical flow as a video stream.

---

## **9. Loop Control**

### Code:
```python
k = cv2.waitKey(30) & 0xff
if k == 27:
    break
```

- **`cv2.waitKey`**: Waits for 30 ms to check for a key press.
- **`27`**: ASCII code for the `Esc` key. If pressed, the loop terminates.

---

## **10. Updating for the Next Iteration**

### Code:
```python
prvsImg = nextImg
```

- Updates `prvsImg` to the current frame (`nextImg`) for the next optical flow calculation.

---

## **11. Cleanup**

### Code:
```python
cap.release()
cv2.destroyAllWindows()
```

- **`cap.release()`**: Releases the video capture resource.
- **`cv2.destroyAllWindows()`**: Closes all OpenCV windows.

---

## **Summary**
1. Captures video from the camera.
2. Converts frames to grayscale for processing.
3. Computes dense optical flow using the Farneback method.
4. Visualizes motion as color-coded flow based on direction and magnitude.
5. Repeats for each frame until the `Esc` key is pressed.
6. Cleans up resources and exits.